In [4]:
# Sample user submission
message = '''
    Create a count for each character in the string.
    For each character:
    Convert it to lowercase.
    Count how many times it appears.
    Set total_length to 0 and odd_count to 0.
    For each character count:
    Add the even part of the count to total_length.
    If the count is odd, add 1 to odd_count.
    If odd_count is greater than 0, add 1 to total_length.
    Return total_length.
'''


In [5]:
# Load JSON data
import json

with open('sample_problem.json', 'r') as file:
    problem_data = json.load(file)

# Define system_prompt
system_prompt = f'''
    You are a professor who specializes in solving code problems.
    The user will submit in words their approach to solving the problem.
    Analyze the user's submission and provide feedback.

    Output your feedback in JSON format with the following structure:
    {{
        'title': 'Create a title for this analysis',
        'analysis': 'Create a detailed analysis on the user's submission',
        'summary': 'Create a concise one-paragraph summary of the analysis',
        'score': 'An integer score: 1 (incorrect), 2 (partially correct), 3 (correct)'
    }}

    Here are the problem details:
    Category: 
    {problem_data['category']}

    Subcategory: 
    {problem_data['subcategory']}

    Difficulty:
    {problem_data['difficulty']}

    Title: 
    {problem_data['title']}

    Description: 
    {problem_data['description']}

    Constraints: 
    {chr(10).join([f'{constraint}' for constraint in problem_data['constraints']])}

    Examples:
    {chr(10).join([f'Input: {example['input']}, Output: {example['output']}, Explanation: {example['explanation']}' for example in problem_data['examples']])}

    Here is the user's submission:
    {message}

    Ensure your output is coherent and follows the JSON structure provided.
'''

print(system_prompt)



    You are a professor who specializes in solving code problems.
    The user will submit in words their approach to solving the problem.
    Analyze the user's submission and provide feedback.

    Output your feedback in JSON format with the following structure:
    {
        'title': 'Create a title for this analysis',
        'analysis': 'Create a detailed analysis on the user's submission',
        'summary': 'Create a concise one-paragraph summary of the analysis',
        'score': 'An integer score: 1 (incorrect), 2 (partially correct), 3 (correct)'
    }

    Here are the problem details:
    Category: 
    manipulations

    Subcategory: 
    strings

    Difficulty:
    easy

    Title: 
    Longest Palindrome

    Description: 
    Given a string s which consists of lowercase or uppercase letters, return the length of the longest palindrome that can be built with those letters. Letters are case sensitive, for example, 'Aa' is not considered a palindrome.

    Constraints: 

In [6]:
# Load Meta-Llama-3.1-8B
from accelerate import disk_offload
from dotenv import load_dotenv
from huggingface_hub import login
import os
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from torch import backends, float32

# Load environment variables and Hugging Face token
load_dotenv()
login(token=os.getenv('HUGGINGFACE_ACCESS_TOKEN'), add_to_git_credential=True)

model_id = 'meta-llama/Meta-Llama-3.1-8B-Instruct'

# Load the model directly, which handles the weights
model = AutoModelForCausalLM.from_pretrained(
    model_id, 
    torch_dtype=float32, 
    trust_remote_code=True, 
    low_cpu_mem_usage=True
)

# Apply disk offloading
disk_offload(model=model, offload_dir='disk_offload')

tokenizer = AutoTokenizer.from_pretrained(model_id)
device = 'mps' if backends.mps.is_available() else 'cpu'

model.to(device)

# Create the inference pipeline
text_generation_pipeline = pipeline(
    'text-generation',
    model=model,
    tokenizer=tokenizer,
    model_kwargs={'torch_dtype': float32},
    # device_map='auto',
    device=0 if device == 'mps' else -1  # Use 0 for MPS, -1 for CPU
)

messages = [
    {'role': 'system', 'content': 'You are a pirate chatbot who always responds in pirate speak!'},
    {'role': 'user', 'content': 'Who are you?'},
]

# Ensure model weights are loaded before inference
outputs = text_generation_pipeline(
    messages,
    max_new_tokens=256,
    pad_token_id = tokenizer.eos_token_id
)

print(outputs[0]['generated_text'])


Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (osxkeychain).
Your token has been saved to /Users/jjpark987/.cache/huggingface/token
Login successful


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

NotImplementedError: Cannot copy out of meta tensor; no data!

In [15]:
# Load LLaMA-2
# from gradio_client import Client

# client = Client('huggingface-projects/llama-2-13b-chat')
# try:
# 	result = client.predict(
# 		message=message,
# 		system_prompt=system_prompt,
# 		max_new_tokens=1024,
# 		temperature=0.6,
# 		top_p=0.9,
# 		top_k=50,
# 		repetition_penalty=1.2,
# 		api_name='/chat'
# 	)
# except Exception as e:
# 	print(f'An error occurred: {e}')

# print(result)
